<a href="https://colab.research.google.com/github/JisnaP/Agentic_AI_Powered_Chatbot_for_Queries_Related_to_CLAT_2025/blob/main/Agentic_AI_powered_chatbot_for_CLAT_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
%pip install tavily-python
!pip install sentence-transformers

In [30]:
!pip install llama_index

In [31]:
import llama_index

In [53]:
import os


In [54]:
os.environ["OPENAI_KEY"] = "***"

In [55]:
OPENAI_KEY = os.getenv("OPENAI_KEY")

In [56]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_KEY)

In [57]:
os.environ["TVLY_API"]="****"

In [58]:
import os
import asyncio
from tavily import AsyncTavilyClient
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.memory import ChatMemoryBuffer


TVLY_API = os.getenv("TVLY_API")
OPENAI_KEY = os.getenv("OPENAI_KEY")


if not TVLY_API:
    raise ValueError("Tavily API key not found. Please set the TVLY_API environment variable.")
if not OPENAI_KEY:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

async def search_web(query: str) -> str:
    """Search the web for information and return results with limited content."""
    try:
        client = AsyncTavilyClient(api_key=TVLY_API)
        results = await client.search(query, include_answer=True, include_raw_content=False)


        search_result = results.get('answer', '') + "\n\nSources:\n"


        for i, result in enumerate(results.get('results', [])[:3]):
            search_result += f"- {result.get('title')}: {result.get('url')}\n"
            # Limit content to ~200 chars per source
            if 'content' in result:
                search_result += f"  {result.get('content')[:200]}...\n"

        return search_result
    except Exception as e:
        return f"Error searching the web: {str(e)}"




In [59]:
llm = OpenAI(
    model="gpt-3.5-turbo",
    api_key=OPENAI_KEY,
    max_tokens=4096,  # Limit output tokens
    temperature=0.7
)

# Create the search tool
tavily_search_tool = FunctionTool.from_defaults(
    fn=search_web,
    name="web_search",
    description="Use this tool to search the web for recent information. Always use this tool when asked about current events, dates, or specific facts."
)
system_prompt_web= (
    "You are a helpful assistant that uses web search to find the most recent and relevant information about CLAT 2025."
     "Use the WebSearch tool to find answers that are not available in official documents — such as current news, announcements, coaching advice, student experiences, preparation tips, or anything dynamic or updated online."
    "If the query relates to official syllabus, exam dates, eligibility, or brochure info, redirect the query to the agent handling official documents instead."
)



# Create memory
memory = ChatMemoryBuffer.from_defaults(token_limit=4000)

# Create agent
web_agent = FunctionAgent(
    name="websearch_agent",
    description = "searches web to answer queries with accurate up-to-date answers",
    tools=[tavily_search_tool],
    llm=llm,
    system_prompt=system_prompt_web,
    memory=memory,
    verbose=False
)



In [60]:
!pip install llama_index_readers_web

In [61]:
from llama_index.readers.web import AgentQLWebReader
agentql_reader = AgentQLWebReader(
    api_key="****",  # Replace with key from https://dev.agentql.com
    params={
        "is_scroll_to_bottom_enabled": True,

    }
)
clat_url = "https://consortiumofnlus.ac.in/clat-2025/"


clat_document = agentql_reader.load_data(
    url=clat_url,
    prompt="Extract all text visible on the page, including syllabus details, important announcements, and content inside dropdown menus or expandable sections."
)



INFO:httpx:HTTP Request: POST https://api.agentql.com/v1/query-data "HTTP/1.1 200 OK"


In [40]:
%pip install llama-index-vector-stores-chroma

In [41]:

%pip install llama-index-embeddings-huggingface

In [62]:
from llama_index.core import VectorStoreIndex
#from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from chromadb import PersistentClient

chroma_client = PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("clat_2025_collection")


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']


In [70]:
from llama_index.core.node_parser import SentenceSplitter
#from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent.workflow import ReActAgent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os

os.environ["HF_TOKEN"] = "****"
HF_TOKEN = os.getenv("HF_TOKEN")
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
#embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=100, chunk_overlap=20),

        embed_model,
    ],
    vector_store=vector_store,
)

# Ingest directly into a vector db
pipeline.run(documents=clat_document)

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)

llm = OpenAI(
    model="gpt-3.5-turbo",
    api_key=OPENAI_KEY,
    max_tokens=4096,  # Limit output tokens
    temperature=0.7
)
index.storage_context.persist("clat_2025_index")
query_engine = index.as_query_engine(llm=llm)
query_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="CLAT2025OfficialDocs",
    description="Best used when the query relates to information from the official CLAT 2025 website, and when semantic search provides a confident, relevant match from the official documents."
)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']


In [77]:
system_prompt_query=(
    "You are an expert assistant with access to official CLAT 2025 documentation."
    "Use the CLAT2025OfficialDocs tool to answer questions about the official syllabus, exam pattern, eligibility criteria, important dates, fees, and other information published on the official CLAT website or brochures"
    "Do not attempt to answer questions that require up-to-date news, coaching advice, or discussion-based content — those should be handled by the web search agent."

)
query_agent=ReActAgent(
    name="Rag_Agent",
    description="Uses vector database for answering queries related to CLAT2025 where semantic search of the vector database gives accurate answers.",
    llm=llm,
    system_prompt=system_prompt_query,
    memory=memory,
    verbose=False
)

In [112]:
# Create workflow
from llama_index.core.workflow import Context
workflow = AgentWorkflow(agents=[web_agent,query_agent], root_agent="Rag_Agent")





In [86]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("Give me last year’s cut-off for NLSIU Bangalore.")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The NLSIU Bangalore cut-off for CLAT 2024 was 102 for the General category. For 2025, the expected cut-off is 95+ marks for General. NLSIU Bangalore has the highest cut-offs among all NLUs.


In [87]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what is the eligibility criteria for CLAT2025")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The eligibility criteria for CLAT 2025 include the following:
- There is no upper age limit.
- Candidates must have completed 10+2 or an equivalent examination.
- There is no specific minimum marks requirement for eligibility.

Sources:
- [Law Prep Tutorial Patna - CLAT Eligibility Criteria 2025](https://www.lawpreptutorialpatna.com/clat-eligibility-criteria/)
- [Consortium of NLUs - CLAT 2025 PG Eligibility Criteria](https://consortiumofnlus.ac.in/clat-2025/pg-eligibility.html)
- [SATHEE - CLAT Eligibility Criteria](https://clat.iitk.ac.in/sathee-clat/clat-exams/clat-ug-2025/clat-details/eligibility-criteria/)


In [88]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what is the eligibility criteria for CLAT2025 in the official website for CLAT2025")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The eligibility criteria for CLAT 2025, as per the official CLAT website, are as follows:
- Candidates must have completed 10+2 or equivalent.
- A minimum of 45% marks is required for the general category, while 40% is required for SC/ST candidates.
- There is no specified age limit mentioned for CLAT 2025.
- More details about the application process will be announced later.

For more information, you can refer to the official CLAT website.


In [106]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("What is the syllabus for CLAT2025")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The CLAT 2025 syllabus includes five subjects: English Language, Legal Reasoning, Logical Reasoning, Current Affairs, and Quantitative Techniques. Preparation requires thorough study of these areas. The exam assesses aptitude and skills for legal education.

Sources:
- CLAT Syllabus 2025 PDF Download For UG and PG - Adda247: [Link](https://www.adda247.com/school/clat-syllabus/)
- CLAT 2025 UG Syllabus - Consortium of NLUs: [Link](https://consortiumofnlus.ac.in/clat-2025/ug-syllabus.html)
- CLAT 2025 Syllabus: Subject-wise Syllabus for CLAT Exam - Careers360: [Link](https://law.careers360.com/articles/clat-syllabus-2025)


In [109]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("when is CLAT 2025 supposed to take place")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The CLAT 2025 exam is scheduled to take place on December 1, 2024. Registration for the exam will be open from July 1 to November 10, 2024. The duration of the exam is two hours.

If you need more information or assistance regarding CLAT 2025, feel free to ask!


In [111]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("when is next CLAT exam after 2025 supposed to take place")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The next CLAT exam after 2025 is scheduled to take place on December 1, 2025. Registration for the exam opens in July 2024. For more information, you can visit the official website at consortiumofnlus.ac.in.


In [113]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what are main updates about CLAT2025 on the official website")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.tavily.com/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The main update about CLAT 2025 on the official website is that the application process will open soon. For the latest information, you can visit the official website at clat.ac.in.


In [114]:
import asyncio
import nest_asyncio


nest_asyncio.apply()


async def main():
    try:
        response = await workflow.run("what is the syllabus for Dec CLAT 2025 on official CLAT 2025 website")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

    pass


if __name__ == "__main__":
    asyncio.run(main())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The syllabus for the December CLAT 2025 exam can typically include topics like English including Comprehension, General Knowledge & Current Affairs, Mathematics, Legal Aptitude, and Logical Reasoning. For the most accurate and up-to-date information, it's best to visit the official CLAT 2025 website.
